# Lab 4 - Model Deplyoment 

In this lab, you will learn how to use Azure Machine Learning Service to deploy, manage, and monitor the trained models.


The following diagram illustrates the complete deployment workflow.

![AML Arch](https://github.com/jakazmie/images-for-hands-on-labs/raw/master/model-ci-cd.png)



The deployment workflow includes the following steps:

- Create/Retrain the model
- Register the model in a registry hosted in your Azure Machine Learning Service workspace
- Register an image that pairs a model with a scoring script and dependencies in a portable container
- Deploy the image as a web service in the cloud or to edge devices
- Monitor and collect data


You completed the first two steps in the previous labs.

In this lab we will walk-through the reminder of the deployment workflow.



## Connect to the workspace

In [1]:
# Verify AML SDK Installed
# view version history at https://pypi.org/project/azureml-sdk/#history 
import azureml.core
print("SDK Version:", azureml.core.VERSION)

SDK Version: 1.0.2


In [2]:
from azureml.core import Workspace

# Read the workspace config from file
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

Found the config file in: /data/home/demouser/notebooks/MTC_AzureAILabs/ML-AML-Walkthrough/aml_config/config.json
jkamlworkshop
jkamlworkshop
southcentralus
952a710c-8d9c-40c1-9fec-f752138cc0b3


## Download and test version of a model from Azure Machine Learning.

Download the model registered in the previous step and run a quick test.

The model's name can be retrieved from *Model Registry* using Azure Portal.

In [7]:
from azureml.core.model import Model
import pickle
import azureml.train.automl

# Download the model to a local directory

model_name = 'AutoMLba62f60a3best'
model_path = Model.get_model_path(model_name, _workspace=ws)
model = pickle.load(open(model_path, 'rb'))
print(model)

azureml-models/AutoMLba62f60a3best/1/model.pkl
Pipeline(memory=None,
     steps=[('datatransformer', DataTransformer(logger=None, task=None)), ('prefittedsoftvotingclassifier', PreFittedSoftVotingClassifier(classification_labels=None,
               estimators=[('LogisticRegression', Pipeline(memory=None,
     steps=[('standardscalerwrapper', <automl.client.core.common.mo... weights=[0.5333333333333333, 0.2, 0.06666666666666667, 0.13333333333333333, 0.06666666666666667]))])


/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator Imputer from version 0.19.1 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.19.1 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator StandardScaler from version 0.19.1 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.19.1 when using version 0.20.2. This might lead to breaking code or inva

In [ ]:
# Re-load the model
model = pickle.load(open(os.path.join(model_path, model_name + '.pkl'), 'rb'))
print("Loaded model from:", os.path.join(model_path, model_name + '.pkl'))

# Re-load the scaler
scaler = pickle.load(open(os.path.join(model_path, scaler_name + '.pkl'),'rb'))
print("Loaded scaler from:", os.path.join(model_path, scaler_name + '.pkl'))

# Run a quick test
age = 60
km = 40000

scaled_input = scaler.transform([[age, km]])
prediction = model.predict(scaled_input)
print(prediction)

## Create and deploy the container image encapsulating the model

When you deploy a model using AML to either ACI or AKS, you are deploying a Docker container encapsulating a trained model, its dependencies, and a web services wrapper around the model. 

### Create a Conda dependencies environment file.


In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 

mycondaenv = CondaDependencies.create(conda_packages=['scikit-learn','numpy','pandas'])

with open("mydeployenv.yml","w") as f:
    f.write(mycondaenv.serialize_to_string())

Review the content of 'yml' file.

In [ ]:
with open("mydeployenv.yml","r") as f:
    print(f.read())

### Create scoring script.
With Azure Machine Learning, you have full control over the logic of the webservice which includes how it loads your model, transforms web service inputs, uses the model for scoring and returns the result. 

In [ ]:
%%writefile score.py

import json
import os
import numpy as np
import pandas as pd
from sklearn import linear_model 
from sklearn.externals import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from azureml.core import Run
from azureml.core import Workspace
from azureml.core.run import Run
from azureml.core.experiment import Experiment
import pickle
from sklearn.externals import joblib

def init():
    try:
        # One-time initialization of predictive model and scaler
        from azureml.core.model import Model
        
        global trainedModel   
        global scaler

        model_name = "usedcarsmodel" 
        model_path = Model.get_model_path(model_name)
        print('Looking for models in: ', model_path)

        trainedModel = pickle.load(open(os.path.join(model_path,'usedcarsmodel.pkl'), 'rb'))
        
        scaler = pickle.load(open(os.path.join(model_path,'scaler.pkl'),'rb'))

    except Exception as e:
        print('Exception during init: ', str(e))

def run(input_json):     
    try:
        inputs = json.loads(input_json)

        #Scale the input
        scaled_input = scaler.transform(inputs)
        
        #Get the scored result
        prediction = json.dumps(trainedModel.predict(scaled_input).tolist())

    except Exception as e:
        prediction = str(e)
    return prediction


### Create docker image for deployment

To create a Container Image, you need four things: the model metadata (as retrieved from Model Registry), the scoring script file, the runtime configuration (defining whether Python or PySpark should be used) and the Conda Dependencies file.

In [ ]:
from azureml.core.image import ContainerImage, Image

# Retrieve the model metadata from Model Registry
model_name = 'usedcarsmodel'
model = Model(workspace=ws, name=model_name)

# Define runtime
runtime = "python" 

# Define scoring script
driver_file = "score.py"

# Define conda dependencies
conda_file = "mydeployenv.yml"

# configure the image
image_config = ContainerImage.image_configuration(execution_script=driver_file, 
                                                  runtime=runtime, 
                                                  conda_file=conda_file,
                                                  description="Image for used cars predictor",
                                                  tags={"Classifier": "Logistic regression"})

image = Image.create(name = "used-car-classifier-image",
                     models = [model],
                     image_config = image_config, 
                     workspace = ws)

image.wait_for_creation(show_output = True)


### Deploy the container image to ACI

With the Container Image  in hand, you are almost ready to deploy to ACI. The next step is to define the size of the VM that ACI will use to run your Container.

In [ ]:
from azureml.core.webservice import AciWebservice, Webservice

aci_config = AciWebservice.deploy_configuration(
    cpu_cores = 1, 
    memory_gb = 1, 
    tags = {'name':'Azure ML ACI'}, 
    description = 'This is a deployment of the affordibility predictor.')

At this point you can deploy the image to the webservice to ACI

In [ ]:
from azureml.core.webservice import Webservice

service_name = "usedcarsmlservice-aci"
print("Deploying: ", service_name)
aci_service = Webservice.deploy_from_image(deployment_config = aci_config,
                                           image = image,
                                           name = service_name,
                                           workspace = ws)
aci_service.wait_for_deployment(True)

### Test the service

Once the webservice deployment completes, you can use the returned webservice object to invoke the webservice. 

In [ ]:
import json
age = 60
km = 40000
test_data  = json.dumps([[age,km]])
test_data
result = aci_service.run(input_data=test_data)
print(result)

### Deploy the container image to AKS

Once you are familiar with the process for deploying a webservice to ACI, you will find the process for deploying to AKS to be similar with one additional step that creates the AKS cluster first.

In [ ]:
# Provision an AKS cluster 

from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import Webservice, AksWebservice

# Use the default configuration, overriding the default location to a known region that supports AKS
prov_config = AksCompute.provisioning_configuration(location='westus2')

aks_name = 'aks-cluster01' 

# Create the cluster
aks_target = ComputeTarget.create(workspace = ws, 
                                  name = aks_name, 
                                  provisioning_configuration = prov_config)


# Wait for cluster to be ready
aks_target.wait_for_completion(show_output = True)
print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)

With your AKS cluster ready, now you can deploy your webservice. Once again, you need to provide a configuration for the size of resources allocated from the AKS cluster to run instances of your Container.

In [ ]:
# Create the web service configuration (using defaults)
aks_config = AksWebservice.deploy_configuration()

aks_service_name ='usedcarsmlservice-aks'

aks_service = Webservice.deploy_from_image(
  workspace=ws, 
  name=aks_service_name, 
  image = image,
  deployment_target=aks_target
  )


aks_service.wait_for_deployment(show_output = True)
print(aks_service.state)

### Test the service
As before, you can use the webservice object returned by the deploy_from_model method to invoke your deployed webservice. 

In [ ]:
import json
age = 60
km = 40000
test_data  = json.dumps([[age,km]])
test_data
result = aks_service.run(input_data=test_data)
print(result)

## Clean up

Make sure to remove ACI and AKS deployments. Use Azure Portal to remove *Deployments* and *AKS Compute*.